# Tutorials

The following provides a short tutorial on how to use some of the main features of Yadism. After this tutorial, the user will know how to:
- prepare the observable and theory cards required for any `yadism` computation
- compute an observable by convoluting `yadism` prediction to a given PDF set
- tabulate the `yadism` predictions into an interpolation table for a later use

## Installation 

In order to start using Yadism, one first needs to install it. The easiest way to install the latest stable version is via `pip` using the following command

```sh
pip install yadism
```

If one wishes in addition to also install the Yadism benchmark pacakge (see yadmark), one can instead run the following command:

```sh
pip install 'yadism[mark]'
```

We can check that Yadism was properly installed by checking the version:

In [1]:
import yadism
yadism.__version__

'0.0.0'

## Preparing the run cards

Yadism takes as inputs two runcards (represented as dictionaries):
- an observable card which contains the details on the observable to be computed
- a theory card which contains the various theory settings

Below is an example of an observable card in which the meaning of each key is also specified:

In [2]:
observable_card = {
    # Process type: "EM", "NC", "CC"
    "prDIS": "NC",
    # Projectile: "electron", "positron", "neutrino", "antineutrino"
    "ProjectileDIS": "electron",
    # Scattering target: "proton", "neutron", "isoscalar", "lead", "iron", "neon" or "marble"
    "TargetDIS": "proton",
    # Interpolation: if True use log interpolation
    "interpolation_is_log": True,
    # Interpolation: polynomial degree, 1 = linear, ...
    "interpolation_polynomial_degree": 4,
    # Interpolation: xgrid values
    # Note: for illustrative purposes the grid is chosen very small here
    "interpolation_xgrid": [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
    # Observables configurations
    "observables": {
        "XSHERANCAVG_charm": [
            {
                "y": 0.8240707777909629,
                "x": 3e-05,
                "Q2": 2.5,
            },
            {
                "y": 0.3531731904818413,
                "x": 7e-05,
                "Q2": 2.5,
            },
            # Add here the kinematics of other datapoints
        ],
        # Potentially include observables other than XSHERANCAVG_charm,
        # each of them has to be: TYPE_heaviness, where heaviness can take:
        # "charm", "bottom", "top", "total" or "light".
    },
    # Projectile polarization faction, float from 0 to 1.
    "PolarizationDIS": 0.0,
    # Exchanged boson propagator correction
    "PropagatorCorrection": 0.0,
    # Restrict boson coupling to a single parton ? Monte Carlo PID or None for all partons
    "NCPositivityCharge": None,
}

Similarly, below is an example of a theory card:

In [3]:
theory_card = {
    "CKM": "0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152",  # CKM matrix elements
    "FNS": "FFNS",  # Flavour Number Scheme, options: "FFNS", "FFN0", "ZM-VFNS"
    "GF": 1.1663787e-05,  # [GeV^-2] Fermi coupling constant
    "IC": 1,  # 0 = perturbative charm only, 1 = intrinsic charm allowed
    "MP": 0.938,  # [GeV] proton mass
    "MW": 80.398,  # [GeV] W boson mass
    "MZ": 91.1876,  # [GeV] Z boson mass
    "NfFF": 4,  # (fixed) number of running flavors, only for FFNS or FFN0 schemes
    "PTO": 2,  # perturbative order in alpha_s: 0 = LO (alpha_s^0), 1 = NLO (alpha_s^1) ...
    "Q0": 1.65,  # [GeV] reference scale for the flavor patch determination
    "nf0": 4,  # number of active flavors at the Q0 reference scale
    "Qref": 91.2,  # [GeV] reference scale for the alphas value
    "nfref": 5,  # number of active flavors at the reference scale Qref
    "alphas": 0.118,  # alphas value at the reference scale
    "TMC": 1,  # include target mass corrections: 0 = disabled, 1 = leading twist, 2 = higher twist approximated, 3 = higher twist exact
    "XIF": 1.0,  # ratio of factorization scale over the hard scattering scale
    "XIR": 1.0,  # ratio of renormalization scale over the hard scattering scale
    "alphaqed": 0.007496252,  # alpha_em value
    "kcThr": 1.0,  # ratio of the charm matching scale over the charm mass
    "kbThr": 1.0,  # ratio of the bottom matching scale over the bottom mass
    "ktThr": 1.0,  # ratio of the top matching scale over the top mass
    "mc": 1.51,  # [GeV] charm mass
    "mb": 4.92,  # [GeV] bottom mass
    "mt": 172.5,  # [GeV] top mass
    "n3lo_cf_variation": 0,  # N3LO coefficient functions variation: -1 = lower bound, 0 = central , 1 = upper bound
    "QED": 0,  # QED correction to running of strong coupling: 0 = disabled, 1 = allowed
    "MaxNfAs": 5,  # maximum number of flavors in running of strong coupling
    "HQ": "POLE",  # heavy quark mass scheme (not yet implemented in yadism)
    "MaxNfPdf": 5,  # maximum number of flavors in running of PDFs (ignored by yadism)
    "ModEv": "EXA",  # evolution solver for PDFs (ignored by yadism)
}

Note that a template observable is provided by `yadmark` (assuming it was installed). One can thus just start from the default card and overwrite the values.

In [4]:
from rich.pretty import pprint # already installed as a dependency, only needed for a pretty printing
from yadmark.data.observables import default_card
pprint(default_card)

{
│   'NCPositivityCharge': None,
│   'PolarizationDIS': 0,
│   'ProjectileDIS': 'electron',
│   'PropagatorCorrection': 0,
│   'TargetDIS': 'proton',
│   'interpolation_is_log': True,
│   'interpolation_polynomial_degree': 4,
│   'interpolation_xgrid': [
│   │   1e-07,
│   │   1.6102620275609392e-07,
│   │   2.592943797404667e-07,
│   │   4.1753189365604003e-07,
│   │   6.723357536499335e-07,
│   │   1.0826367338740541e-06,
│   │   1.7433288221999873e-06,
│   │   2.8072162039411756e-06,
│   │   4.520353656360241e-06,
│   │   7.2789538439831465e-06,
│   │   1.1721022975334793e-05,
│   │   1.8873918221350995e-05,
│   │   3.039195382313195e-05,
│   │   4.893900918477499e-05,
│   │   7.880462815669905e-05,
│   │   0.0001268961003167922,
│   │   0.00020433597178569417,
│   │   0.00032903445623126676,
│   │   0.0005298316906283707,
│   │   0.0008531678524172806,
│   │   0.0013738237958832637,
│   │   0.00221221629107045,
│   │   0.003562247890262444,
│   │   0.005736152510448681,
│   │   0.009236708571873866,
│   │   0.014873521072935119,
│   │   0.02395026619987486,
│   │   0.03856620421163472,
│   │   0.06210169418915616,
│   │   0.1,
│   │   0.1473684210526316,
│   │   0.19473684210526315,
│   │   0.24210526315789474,
│   │   0.2894736842105263,
│   │   0.33684210526315794,
│   │   0.38421052631578945,
│   │   0.43157894736842106,
│   │   0.4789473684210527,
│   │   0.5263157894736842,
│   │   0.5736842105263158,
│   │   0.6210526315789474,
│   │   0.6684210526315789,
│   │   0.7157894736842105,
│   │   0.7631578947368421,
│   │   0.8105263157894737,
│   │   0.8578947368421053,
│   │   0.9052631578947369,
│   │   0.9526315789473684,
│   │   1.0
│   ],
│   'observables': {},
│   'prDIS': 'EM'
}

## Computing predictions

Now that we have all the necessary inputs, we can now compute the corresponding predictions:

In [12]:
out = yadism.run_yadism(theory_card, observable_card)

[17:13:31] INFO     Interpolation: number of points = 8, polynomial degree = 4, logarithmic =  ]8;id=855744;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=350159;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/interpolation.py#537\537]8;;\
                    True                                                                                           

           INFO     {'MZ2': 8315.178393760001, 'CKM': CKM([[9.49221518e-01 5.07600900e-02  ]8;id=209628;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/coefficient_functions/coupling_constants.py\coupling_constants.py]8;;\:]8;id=423060;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/coefficient_functions/coupling_constants.py#52\52]8;;\
                    1.20409000e-05] [5.07150400e-02 9.47604903e-01 1.68100000e-03]                                 
                    [7.43044000e-05 1.62409000e-03 9.98304719e-01]]), 'sin2theta_weak':                            
                    0.23121, 'MW2': 6463.838403999999}                                                             

           INFO     {'process': 'NC', 'projectilePID': 11, 'polarization': 0.0,            ]8;id=742509;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/coefficient_functions/coupling_constants.py\coupling_constants.py]8;;\:]8;id=926420;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/coefficient_functions/coupling_constants.py#53\53]8;;\
                    'propagatorCorrection': 0.0, 'nc_pos_charge': None}                                            

           INFO     RenScaleVar: False, FactScaleVar: False                                  ]8;id=321435;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=492824;file:///home/tanjona/Documents/WORK/NNPDF/yadism/src/yadism/esf/scale_variations.py#61\61]8;;\

           INFO     Atlas [0.00e+00 - 0.00e+00 - inf - inf - inf], ref=2.7224999999999997 @ 4       ]8;id=853001;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=372449;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py#69\69]8;;\

...

Plan

 • XSHERANCAVG_charm at 2 pts

Calculation

yadism took off! please stay tuned ...

took 0.23 s

`out` is an object of type `yadism.output.Output` from which one can perform various operations. In our example, we are interested in convoluting it with a PDF set in the LHAPDF format.

To do so, first we need to import the `lhapdf` package. For this example, we will use the NNPDF4.0 set: `NNPDF40_nnlo_as_01180`.

In [13]:
import lhapdf
pdf = lhapdf.mkPDF("NNPDF40_nnlo_as_01180")

LHAPDF 6.5.0 loading /home/tanjona/miniconda3/envs/yadism/share/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331100


The convolution can simply be performed as follow:

In [14]:
values = out.apply_pdf(pdf)

[17:13:33] INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=295909;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=914974;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py#69\69]8;;\

           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=48542;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=745971;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/couplings.py#458\458]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=596034;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=509149;file:///home/tanjona/miniconda3/envs/yadism/lib/python3.10/site-packages/eko/matchings.py#69\69]8;;\

In [15]:
pprint(values)

{
│   'XSHERANCAVG_charm': [
│   │   {
│   │   │   'x': 3e-05,
│   │   │   'Q2': 2.5,
│   │   │   'result': -0.9959496128078331,
│   │   │   'error': 1.281070214968848e-08,
│   │   │   'y': 0.8240707777909629
│   │   },
│   │   {
│   │   │   'x': 7e-05,
│   │   │   'Q2': 2.5,
│   │   │   'result': -0.7710853256052412,
│   │   │   'error': 2.3298246570353407e-08,
│   │   │   'y': 0.3531731904818413
│   │   }
│   ]
}

## Dumping (loading) predictions into (from) fast-interpolation grids

One of the main advantages of `yadism` is its interface with [PineAPPL](https://github.com/NNPDF/pineappl) - a library providing fast-interpolation grids. This is quite convenient in that one can re-use the same computations without the need to re-run `yadism`.

Each observable defined in `observable_card` can be dumped into its own fast-interpolation grid. The steps are as follow:

In [16]:
from yadbox.export import dump_pineappl_to_file

# Extract the names of the different observables (in case there are many)
list_observables = observable_card['observables'].keys()

# Dump each observable into a PineAPPL grid
for obs in list_observables:
    dump_pineappl_to_file(out, f"outputgrid_{obs}.pineappl.lz4", f"{obs}")

To load the pre-computed predictions, we need to import the `pineappl` library. `pineappl` is part of the dependency of `yadism` so we don't have to install it.

In [17]:
import pineappl

# Read the pineappl grid saved earlier
grid = pineappl.grid.Grid.read("./outputgrid_XSHERANCAVG_charm.pineappl.lz4")

# Convolute the grid with the same PDF as above
results = grid.convolute_with_one(2212, pdf.xfxQ2, pdf.alphasQ2)

In [18]:
pprint(results)

array([-0.98699882, -0.7640712 ])